In [1]:
# coding: utf-8
import pandas as pd
import numpy as np

In [45]:
year = 2013
keyword = 'IncomeLossFromContinuingOperationsBeforeIncomeTaxesForeign'

# Load data

q1_sub = pd.read_csv(str(year)+'q1sub.tsv',sep='\t', encoding = "ISO-8859-1")
q1_num = pd.read_csv(str(year)+'q1num.tsv',sep='\t', encoding = "ISO-8859-1")    
q2_sub = pd.read_csv(str(year)+'q2sub.tsv',sep='\t', encoding = "ISO-8859-1")
q2_num = pd.read_csv(str(year)+'q2num.tsv',sep='\t', encoding = "ISO-8859-1")
q3_sub = pd.read_csv(str(year)+'q3sub.tsv',sep='\t', encoding = "ISO-8859-1")
q3_num = pd.read_csv(str(year)+'q3num.tsv',sep='\t', encoding = "ISO-8859-1")
q4_sub = pd.read_csv(str(year)+'q4sub.tsv',sep='\t', encoding = "ISO-8859-1")
q4_num = pd.read_csv(str(year)+'q4num.tsv',sep='\t', encoding = "ISO-8859-1")

sub=q1_sub.append(q2_sub).append(q3_sub).append(q4_sub)
num=q1_num.append(q2_num).append(q3_num).append(q4_num)


# # 2018
# q1_sub = pd.read_csv(str(year)+'q1sub.tsv',sep='\t', encoding = "ISO-8859-1")
# q1_num = pd.read_csv(str(year)+'q1num.tsv',sep='\t', encoding = "ISO-8859-1")    
# sub = q1_sub
# num = q1_num

C:\Users\josh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\josh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
num = pd.read_csv(str(2013)+'q2num.tsv',sep='\t', encoding = "ISO-8859-1")  

C:\Users\josh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [67]:
num[num.adsh=='0000060667-13-000045'][num[num.adsh=='0000060667-13-000045'].value==36000000].tag.tolist()

['UndistributedEarningsOfForeignSubsidiaries',
 'TangibleAssetImpairmentCharges']

In [ ]:
['IncomeLossFromContinuingOperationsBeforeIncomeTaxesExtraordinaryItemsNoncontrollingInterest'] 

In [35]:
# Including only the fields that we want
num = num[num.tag.isin([keyword])]

# Type of records we want
sub = sub[sub.form.isin(['10-K'])]
sub.adsh.nunique()

3944

In [36]:
## STEP #1:
# Get values from NUM file using SUM file matching adsh

df = pd.merge(sub,num,on=['adsh'],how='left')
df = df.loc[:,['adsh','cik','name','sic','coreg','fye','fy','qtrs','fp','filed','form','period','ddate','nciks','aciks','tag','value']]
df.sort_values('cik',inplace=True)
len(df)

6607

In [37]:
# Setting unique identifiers for pivoting table
df['coreg'] = df['coreg'].fillna('null')
df['value'] = df['value'].astype('str')
df['adsh+coreg+value'] = df['adsh'] + "_"+ df['coreg'] + "_" + df['value']
# Reset data type numeric for value
df['value'] = df['value'].astype('float64')

# Make a copy for pivoting table
data = df.copy()
data.sort_values(['cik','tag','period'],inplace=True)
data.adsh.nunique()

3944

In [38]:
# Pivoting table
fin_data =data.loc[:,['adsh+coreg+value','adsh','cik','name','sic','qtrs','coreg','fy','fp','form','ddate','filed']].\
merge(pd.pivot_table(data,index=data['adsh+coreg+value'],columns='tag',values='value').reset_index(),on='adsh+coreg+value',how='left')

# Some statistics and info
fin_data.adsh.nunique()

3944

In [39]:
fin_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6607 entries, 0 to 6606
Data columns (total 13 columns):
adsh+coreg+value                                              6607 non-null object
adsh                                                          6607 non-null object
cik                                                           6607 non-null int64
name                                                          6607 non-null object
sic                                                           6606 non-null float64
qtrs                                                          3980 non-null float64
coreg                                                         6607 non-null object
fy                                                            6607 non-null int64
fp                                                            6607 non-null object
form                                                          6607 non-null object
ddate                                                         3

In [40]:
fin_data.to_csv('josh_data_notes_'+str(year)+keyword+'grab_all.csv')